Task 9. 
But, what about a multi-class classifier for MNIST digits? For multi-class linear classification with d classes, one standard approach is to learn a linear mapping f : Rn → Rd where the “y”-value for the ith class is chosen to be the standard basis vector ei ∈ Rd. This is sometimes called one-hot encoding. Using the same A matrix as before and a matrix Y , defined by Yi,j = 1, if observation i in class j and Yi,j = 0 otherwise, we can solve for the coefficient matrix B ∈ Rn×d entries. The classifier maps a vector x to class i if the ith element of BTx is the largest element
in the vector. Report both the overall classification error rate and confusion matrices for the both the training and test sets.

In [1]:
import pandas as pd
import math as math
import numpy as np
import sympy as sy
from sympy import series, Symbol
from sympy.functions import sin, cos, exp
from sympy.plotting import plot
from sympy import lambdify
from numpy import linspace
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
### loads the MNIST training data and plots the first 30 images.
import pandas as pd
import matplotlib.pyplot as plt
# Read MNIST csv file into dataframe
df = pd.read_csv('mnist_train.csv')
#df2 = pd.DataFrame(-255*np.ones(len(df.pixel1)),columns=list('a'))

# to insert -1 prepend
df.insert(1, "a", -1) 
# Merge pixels into feature column and keep only feature
#axis 1 apply function to each row
df ['feature'] = df.apply(lambda row: row.values [1 :], axis =1)
df = df[['feature','label']]
df

,feature,label
0,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
1,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
3,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
4,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
...,...,...
41995,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
41996,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
41997,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7
41998,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",6


In [4]:
import sklearn
from sklearn.model_selection import train_test_split
train, test =train_test_split(df, train_size = 0.5)
train_label=train.label
train_feature=train.feature
test_label=test.label
test_feature=test.feature

In [5]:
## one vs all for train data
A=np.vstack(train_feature)
l=len(A)
Y=np.hstack(train_label)
#print(Y)
# 0 and others
S=[]
for i in range (0,10):
    c=np.where(Y == i, Y, -1)
    c=np.where(c == -1, c, 0)
    c=abs(np.invert(c))#+2
    s=np.linalg.lstsq(A, c,rcond=None)[0]
    S.append(s)
#print(S)
S=np.array(S)

In [20]:
# compute C linear combinations of input point, one per classifier
def model(x,w):
    a = np.dot(x,w.T)+w.T[0]
    return a.T
# the fusion rule
def fusion_rule(x,w):
    return np.argmax(model(x,w))
P=[]
for i in range (0,len(A)):
    cA=np.array(fusion_rule(A[i],S))
    P.append(cA)
P=np.array(P)
CER=np.count_nonzero(P-Y)
rate=np.array(CER)/l
print("classification error rate is",rate)


#matrix
import pandas as pd
data = {'y_Actual':    np.array(Y),
        'y_Predicted': np.array(P)
        }
dfm = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(dfm['y_Actual'], dfm['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

T9_Confusion_Matrix_Train = pd.DataFrame(confusion_matrix)
T9_Confusion_Matrix_Train.to_csv('T9_Confusion_Matrix_Train.csv', index=False)

import scipy.linalg as la
(P, L, U) = la.lu(np.array(confusion_matrix))
#print(P)
#print(L)
#print(U)
D = np.diag(np.diag(U))   # D is just the diagonal of U
U /= np.diag(U)[:, None]  # Normalize rows of U
#print(P.dot(L.dot(D.dot(U))))    # Check
print(U)

T9_U_Train = pd.DataFrame(U)
T9_U_Train.to_csv('T9_UpperTraingle_Train.csv', index=False)

classification error rate is 0.1990952380952381
Predicted     0     1     2     3     4    5     6     7     8     9
Actual                                                              
0          1849     0    14    18     3   12    23     0   137     3
1             0  2067    11    22     0    2     5     0   261     1
2            19    22  1613    80    13    2    87    13   215    19
3             2     9    58  1808     2    6    10    21   223    40
4             4    20    33    16  1453    9    18     3   272   237
5            17     9    19   247     9  822    49    10   615    68
6            26     3    25     2     7   15  1896     0    90     3
7             8    32    23    46    25    2     4  1734    82   258
8             6    27    15    47     2   17    12     3  1817    37
9             6     3    10    32    26    2     0   112   165  1760
[[ 1.00000000e+00  0.00000000e+00  7.57166036e-03  9.73499189e-03
   1.62249865e-03  6.48999459e-03  1.24391563e-02  0.00000

In [21]:
## for test data
A=np.vstack(test_feature)
Y=np.hstack(test_label)
P=[]
for i in range (0,len(A)):
    c=np.array(fusion_rule(A[i],S))
    P.append(c)
P=np.array(P)
CER=np.count_nonzero(P-Y)
rate=CER/l
print("classification error rate is",rate)
#print(CER)
#matrix
import pandas as pd
data = {'y_Actual':    np.array(Y),
        'y_Predicted': np.array(P)
        }
dfm = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(dfm['y_Actual'], dfm['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

import scipy.linalg as la
(P, L, U) = la.lu(np.array(confusion_matrix))

D = np.diag(np.diag(L))   # D is just the diagonal of U
L /= np.diag(L)[:, None]  # Normalize rows of U
#print(P.dot(L.dot(D.dot(U))))    # Check
print(L)

T9_Confusion_Matrix_Test = pd.DataFrame(confusion_matrix)
T9_Confusion_Matrix_Test.to_csv('T9_Confusion_Matrix_Test.csv', index=False)
T9_LT_Test = pd.DataFrame(L)
T9_LT_Test.to_csv('T9_LowerTraingle_Test.csv', index=False)

classification error rate is 0.1990952380952381
Predicted     0     1     2     3     4    5     6     7     8     9
Actual                                                              
0          1849     0    14    18     3   12    23     0   137     3
1             0  2067    11    22     0    2     5     0   261     1
2            19    22  1613    80    13    2    87    13   215    19
3             2     9    58  1808     2    6    10    21   223    40
4             4    20    33    16  1453    9    18     3   272   237
5            17     9    19   247     9  822    49    10   615    68
6            26     3    25     2     7   15  1896     0    90     3
7             8    32    23    46    25    2     4  1734    82   258
8             6    27    15    47     2   17    12     3  1817    37
9             6     3    10    32    26    2     0   112   165  1760
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000